In [1]:
import numpy as np
import random
import csv
import math 
import networkx as nx
import pandas as pd
from operator import itemgetter
from numpy import linalg as LA
import matplotlib.pyplot as plt
from random import uniform, seed
import time
from itertools import zip_longest
G=nx.Graph()
# f=open("fb-pages-sport.txt","r")
# edge1=[]
# edge2=[]
# nodelist=[]
# for x in f:
#     e1,e2=x.split()
#     edge1.append(int(e1)-1)
#     edge2.append(int(e2)-1)
#     for i in (int(e1),int(e2)):
#         if i not in nodelist:
#             nodelist.append(i)
f = open("bio-dmela.txt", "r")
edge1=[]
edge2=[]
nodelist=[]
for x in f:
  e1,e2=x.split()
  edge1.append(int(e1))
  edge2.append(int(e2))
  for i in (int(e1),int(e2)):
      if i not in nodelist:
          nodelist.append(i)
n=len(nodelist)

for  i in range(0, len(edge1)):
    G.add_edge(edge1[i], edge2[i])
# Define a function to run the Independent Cascade Model for one time step
def independent_cascade_step(graph, activated, p):
    # Initialize the set of nodes activated in this time step
    newly_activated = set()
    
    # Loop through all activated nodes
    for node in activated:
        # Loop through the node's neighbors
        for neighbor in graph[node]:
            # If the neighbor is not already activated
            if neighbor not in activated:
                # Generate a random number
                rand = np.random.random()
                # If the number is less than the activation probability
                if rand < p:
                    # Add the neighbor to the set of newly activated nodes
                    newly_activated.add(neighbor)
    
    # Return the set of newly activated nodes
    return newly_activated

# Define a function to run the Independent Cascade Model for a specified number of time steps
def independent_cascade(graph, seeds, p, num_steps, num_sims):
    # Initialize the activated set of nodes with the seeds
    activated = set(seeds)
    # Initialize the list of activated sets for each time step
    activated_lists = [activated]
    countA=[]
    # Loop through the number of Monte Carlo simulations
    for i in range(num_sims):
        # Initialize the activated set of nodes with the seeds
        activated = set(seeds)
        
        # Initialize a list to store the number of activated nodes for each time step
        activated_counts = [len(activated)]
        
        # Loop through the number of time steps
        for j in range(num_steps):
            # Run the Independent Cascade Model for one time step
            newly_activated = independent_cascade_step(graph, activated, p)
            # Add the newly activated nodes to the activated set
            activated.update(newly_activated)
            # Add the number of newly activated nodes to the activated_counts list
            activated_counts.append(len(activated))
        countA.append([x for x in activated_counts])
        # Write the activated_counts list to a file
        #with open(filename, 'w') as f:
        #    f.write(','.join([str(x) for x in activated_counts]) + '\n')
    
    # Return the list of activated sets for each time step
    return activated_lists,countA
seednodes=[214, 662, 707, 75, 60, 1410, 239, 330, 208, 298]
          
beta=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40]

for i in beta:
    act,count=independent_cascade(G,seednodes,i,40, 100)
    #print(count)
    a=np.mean(count, axis=0)
    df = pd.DataFrame.from_dict({'time':[x for x in range(1, len(a)+1)],'cuminf':a})
    df.to_excel('avginfFBfood_ICM_BC_'+str(i)+'.xlsx', header=True, index=False)

# Define a function to run the Independent Cascade Model
def independent_cascadediffbeta(graph, seeds, p):
    # Initialize the active set of nodes with the seeds
    active = set(seeds)
    # Initialize the activated set of nodes with the seeds
    activated = set(seeds)
    
    # Loop until no more nodes can be activated
    while active:
        # Get a random active node
        node = random.choice(list(active))
        # Loop through the node's neighbors
        for neighbor in graph[node]:
            # If the neighbor is not already activated
            if neighbor not in activated:
                # Generate a random number
                rand = np.random.random()
                # If the number is less than the activation probability
                if rand < p:
                    # Add the neighbor to the activated set
                    activated.add(neighbor)
                    # Add the neighbor to the active set
                    active.add(neighbor)
        # Remove the current node from the active set
        active.remove(node)
    
    # Return the set of activated nodes
    return activated

# Define a function to run Monte-Carlo simulations of the Independent Cascade Model
def monte_carlo_independent_cascade(graph, seeds, p, num_simulations):
    # Initialize the list of activated nodes for each simulation
    activated_lists = []
    # Loop through the number of simulations
    for i in range(num_simulations):
        # Run a single Independent Cascade Model simulation
        activated = independent_cascadediffbeta(graph, seeds, p)
        # Append the activated nodes to the list
        activated_lists.append(activated)
    
    # Count the number of times each node was activated across all simulations
    counts = {}
    for activated in activated_lists:
        for node in activated:
            if node in counts:
                counts[node] += 1
            else:
                counts[node] = 1
    
    # Return the dictionary of node counts
    return counts
DiffBeta=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40]

t=np.zeros(len(DiffBeta))
l=0
for i in DiffBeta:
    counts=monte_carlo_independent_cascade(G,seednodes,i,100)
    t[l]=len(counts)
    l=l+1
df = pd.DataFrame.from_dict({'Beta':DiffBeta,'cumulative':t})
df.to_excel('DiffrentBeta_ICM_BC.xlsx', header=True, index=False)